In [1]:
# importing the required libraries
import pandas as pd
import numpy as np
 
# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go

# Forecasting
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

# Coder: Sohel Rana

In [5]:
# main contains file
world_data1 = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')[['date', 'location', 'new_tests', 'total_tests', 'new_cases', 'total_cases', 'new_deaths', 'total_deaths']]
# taking only recovered data
world_data2 = pd.read_csv('https://github.com/datasets/covid-19/raw/master/data/time-series-19-covid-combined.csv')[['Date', 'Country/Region', 'Recovered', 'Confirmed']]

In [9]:
country = input(str('Type a country name: '))

Type a country name: Bangladesh


In [10]:
# For world_data1
df1 = world_data1.query('location == "'+country+'" and total_cases > 0')
df1 = df1.copy().fillna(0)
df1 = df1.rename(columns={'date': 'Date', 'location': 'Country', 'new_tests': 'New_Tests', 'total_tests': 'Tests', 'new_cases':'New_Confirmed', 'total_cases':'Confirmed', 'new_deaths':'New_Deaths', 'total_deaths':'Deaths'})
df1 = df1[['Date', 'Country', 'New_Tests', 'Tests', 'New_Confirmed', 'Confirmed', 'New_Deaths', 'Deaths']]
df1

,Date,Country,New_Tests,Tests,New_Confirmed,Confirmed,New_Deaths,Deaths
4201,2020-03-08,Bangladesh,10.0,137.0,3.0,3.0,0.0,0.0
4202,2020-03-09,Bangladesh,10.0,147.0,0.0,3.0,0.0,0.0
4203,2020-03-10,Bangladesh,16.0,163.0,0.0,3.0,0.0,0.0
4204,2020-03-11,Bangladesh,24.0,187.0,0.0,3.0,0.0,0.0
4205,2020-03-12,Bangladesh,24.0,211.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...
4507,2021-01-08,Bangladesh,15381.0,3328831.0,785.0,520690.0,16.0,7734.0
4508,2021-01-09,Bangladesh,12908.0,3341739.0,692.0,521382.0,22.0,7756.0
4509,2021-01-10,Bangladesh,12920.0,3354659.0,1071.0,522453.0,25.0,7781.0
4510,2021-01-11,Bangladesh,0.0,0.0,849.0,523302.0,22.0,7803.0


In [11]:
# For world_data2
df2 = world_data2.copy()
df2 = df2.rename(columns={'Country/Region': 'Country'})
df2 = df2.query('Country == "'+country+'" and Confirmed > 0')
df2['New_Recovered'] = df2.Recovered.diff().fillna(0).apply(lambda x : x if x > 0 else 0)
df2 = df2[['Date', 'Recovered', 'New_Recovered']]
df2

,Date,Recovered,New_Recovered
6946,2020-03-08,0.0,0.0
6947,2020-03-09,0.0,0.0
6948,2020-03-10,0.0,0.0
6949,2020-03-11,0.0,0.0
6950,2020-03-12,0.0,0.0
...,...,...,...
7240,2020-12-27,451961.0,1473.0
7241,2020-12-28,453318.0,1357.0
7242,2020-12-29,454563.0,1245.0
7243,2020-12-30,456070.0,1507.0


In [12]:
# Concate two files column
df = pd.merge(df1, df2, left_on='Date', right_on='Date').reset_index(drop=True).fillna(0)
df = df[['Date', 'Country', 'New_Tests', 'Tests', 'New_Confirmed', 'Confirmed', 'New_Recovered', 'Recovered', 'New_Deaths', 'Deaths']]
# df.to_csv(r'Bangladeshi-covid.csv', index = False) # download
df

,Date,Country,New_Tests,Tests,New_Confirmed,Confirmed,New_Recovered,Recovered,New_Deaths,Deaths
0,2020-03-08,Bangladesh,10.0,137.0,3.0,3.0,0.0,0.0,0.0,0.0
1,2020-03-09,Bangladesh,10.0,147.0,0.0,3.0,0.0,0.0,0.0,0.0
2,2020-03-10,Bangladesh,16.0,163.0,0.0,3.0,0.0,0.0,0.0,0.0
3,2020-03-11,Bangladesh,24.0,187.0,0.0,3.0,0.0,0.0,0.0,0.0
4,2020-03-12,Bangladesh,24.0,211.0,0.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
294,2020-12-27,Bangladesh,12650.0,3168721.0,1049.0,509148.0,1473.0,451961.0,24.0,7452.0
295,2020-12-28,Bangladesh,12617.0,3181338.0,932.0,510080.0,1357.0,453318.0,27.0,7479.0
296,2020-12-29,Bangladesh,14588.0,3195926.0,1181.0,511261.0,1245.0,454563.0,30.0,7509.0
297,2020-12-30,Bangladesh,15229.0,3211155.0,1235.0,512496.0,1507.0,456070.0,22.0,7531.0


In [13]:
# Comparison with Confirmed and Recovered
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y = df['Confirmed'], mode='lines+markers', name='Confirmed',line=dict(color='Red', width=2)))
fig.add_trace(go.Scatter(x=df['Date'], y = df['Recovered'], mode='lines+markers', name='Recovered',line=dict(color='Blue', width=2)))
fig.update_layout(title='Comparision with Confirmed and Recovered', xaxis_tickfont_size=12, yaxis=dict(title='Cases'), plot_bgcolor='rgb(230, 230, 230)', height=600)
fig.show()

In [14]:
# Last 14 days New Test, Recovered and Confirmed
fig = go.Figure(data=[
    go.Bar(x=df["Date"].tail(14), y=df["New_Tests"].tail(14), name='Test', offsetgroup=0),
    go.Bar(x=df["Date"].tail(14), y=df["New_Recovered"].tail(14), name='Recovered', offsetgroup=0),
    go.Bar(x=df["Date"].tail(14), y=df["New_Confirmed"].tail(14), name='Confirmed', offsetgroup=0)
])
fig.update_layout(title_text='Last 14 days New Test, Recovered and Confirmed', xaxis_tickfont_size=12, yaxis=dict(title='Cases'), plot_bgcolor='rgb(230, 230, 230)')
# fig.update_layout(barmode='group')
fig.show()

In [15]:
Confirmed_days = input('How many days for Confirmed: ')
Confirmed_days = int(Confirmed_days)

Confirmed = df.groupby('Date').sum()['New_Confirmed'].reset_index().tail(Confirmed_days)

How many days for Confirmed: 90


In [16]:
Confirmed.columns = ['ds','y']
Confirmed['ds'] = pd.to_datetime(Confirmed['ds'])
# Confirmed.tail()

m = Prophet(interval_width=0.95)
m.fit(Confirmed)
future = m.make_future_dataframe(periods=7)
# future.tail()

forecast = m.predict(future)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

plot_plotly(m, forecast)
# plot_components_plotly(m, forecast) # trend plot

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
# Deaths
Deaths_days = input('How many days for Deaths: ')
Deaths_days = int(Deaths_days)

Deaths = df.groupby('Date').sum()['New_Deaths'].reset_index().tail(Deaths_days)

How many days for Deaths: 90


In [ ]:
Deaths.columns = ['ds','y']
Deaths['ds'] = pd.to_datetime(Deaths['ds'])
# Deaths.tail()

m = Prophet(interval_width=0.95)
m.fit(Deaths)
future = m.make_future_dataframe(periods=7)
# future.tail()

forecast = m.predict(future)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

plot_plotly(m, forecast)
# plot_components_plotly(m, forecast) # trend plot

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
# Recovered
Recovered_days = input('How many days for Recovered: ')
Recovered_days = int(Recovered_days)

Recovered = df.groupby('Date').sum()['New_Recovered'].reset_index().tail(Recovered_days)

How many days for Recovered: 22


In [ ]:
Recovered.columns = ['ds','y']
Recovered['ds'] = pd.to_datetime(Recovered['ds'])
# Recovered.tail()

m = Prophet(interval_width=0.95)
m.fit(Recovered)
future = m.make_future_dataframe(periods=7)
# future.tail()

forecast = m.predict(future)
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

plot_plotly(m, forecast)
# plot_components_plotly(m, forecast) # trend plot

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 16.
